In [ ]:
import ee
import geemap
import os
import matplotlib.pyplot as plt

import sys
sys.path.append('C:/Users/TEDYqm/Desktop/gitMS_Multimode/GoogleEarthEngine/')

from function import ndvi
from function import ndvi_mean_single
from function import ndvi_dates_values

ee.Authenticate()
os.environ['HTTP_PROXY']='http://127.0.0.1:7890'
os.environ['HTTPS_PROXY']='http://127.0.0.1:7890'
ee.Initialize()

In [ ]:
#边界
china_boundary = ee.FeatureCollection("projects/chancy479/assets/province")
guangxi_cities = ee.FeatureCollection("projects/chancy479/assets/city2")

GX = china_boundary.filterMetadata('ID','equals','Guangxi')
GX_cities=guangxi_cities.filterMetadata('省','equals','广西壮族自治区')


将数据集按每一年分，储存在列表中
列表为：ndvi_datasets

In [ ]:
# 初始化一个空列表，用于存储数据集
ndvi_datasets = []

# 循环获取数据集
for year in range(2000, 2021):
    start_date = str(year) + '-01-01'
    end_date = str(year) + '-12-31'
    
    # 获取数据集并筛选
    ndvi_dataset = ee.ImageCollection('LANDSAT/LE07/C01/T1_8DAY_NDVI')\
        .filterBounds(GX)\
        .filterDate(start_date, end_date)
        
    # 将数据集添加到列表中
    ndvi_datasets.append(ndvi_dataset)


对ndvi_datasets列表中的每一个imagecollection应用qualityMosaic
每一个imagecollection生成一个image
将image添加到列表中，列表为：greenest_datasets

In [ ]:
#获取了每年的最大值图片列表 greenest_datasets
greenest_datasets = []

for ndvi_dataset in ndvi_datasets:
    greenest = ndvi_dataset.qualityMosaic('NDVI')
    greenest_datasets.append(greenest)

计算出列表中每张图片的ndvi平均最大值，返回带有meanNDVI的image，储存在greenest_datasets_meanMax列表中

In [ ]:
# 计算单张图片的平均NDVI
def ndvi_meanMax(image):
    mean = image.reduceRegion(reducer=ee.Reducer.mean(), scale=30, maxPixels=1e9,geometry=GX)#计算所有波段的平均值
    return image.set('meanNDVI', mean.get('NDVI'))

#计算每张图片的平均最大值储存在greenest_datasets_meanMax
greenest_datasets_meanMax = []
for i in range(len(greenest_datasets)):
    meanMax_image = ndvi_meanMax(greenest_datasets[i])
    greenest_datasets_meanMax.append(meanMax_image)
    

In [ ]:
print(len(ndvi_values))

In [ ]:
ndvi_values = []
for i in range(18,21):
    print(str(i))
    mean_ndvi_value = greenest_datasets_meanMax[i].getInfo()['properties']['meanNDVI']
    print("ok!")
    ndvi_values.append(mean_ndvi_value)

获取每张图片的ndvi平均最大值，储存在列表 ndvi_values中

In [ ]:
ndvi_values = []
for i in range(len(greenest_datasets_meanMax)):
    print(str(i))
    mean_ndvi_value = greenest_datasets_meanMax[i].getInfo()['properties']['meanNDVI']
    print("ok!")
    ndvi_values.append(mean_ndvi_value)

把获得的ndvi列表用pickle储存在本地，不用每次都跑一遍

In [ ]:
import pickle
with open('ndvi_values_00_20.pkl', 'wb') as f:
    pickle.dump(ndvi_values, f)

In [ ]:
for i in range(len(ndvi_values)):
    print(ndvi_values[i])